# Surfaces SDF

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import math
import torch
import torchlensmaker as tlm

from torch.nn.functional import normalize

import matplotlib as mpl

Tensor = torch.Tensor

# idea: Q =) F(P+tV) is a 1D function, plot it
# also plot Q' = V . grad F
# plot iterated t values


# 12 plots
# for each surface class
# plot F(x,y) : check finite everywhere
# plot grad_x F (x,y)  : check finite everywhere
# plot grad_y F (x,y)  : check finite everywhere
# plot norm(grad): check non zero
# for few values of V, plot F_grad . V
# V: (0,1), (1,0), (a,a)
#    (0, -1), (-1, 0), (-a, a), (a, -a), (-a, -a)

# check that init_t for any (P,V) does not result in F_grad . V = 0

# Create the input grid tensor
def sample_grid(xlim, ylim, N):
    x = np.linspace(xlim[0], xlim[1], N)
    y = np.linspace(ylim[0], ylim[1], N)
    X, Y = np.meshgrid(x, y)
    return X, Y, torch.tensor(np.stack((X, Y), axis=-1).reshape(-1, 2))

def surface_sdf_analysis(surface, xlim=(-10, 10), ylim=(-10, 10)):
    "Make analysis plots of f and f_grad for an implicit surface"

    # 8 subplots
    f, axes = plt.subplots(2, 4, figsize=(18, 24))

    # F, grad_x F, grad_y F, ||grad F||
    (ax_f, ax_grad_arg, ax_grad_norm, _) = axes[0]

    # axes for (F_grad . V) for 8 values of V
    axes_V = axes.flat[4:9]

    # Create the input grid tensor
    X, Y, points = sample_grid(xlim, ylim, 250)

    # The 4 V values of interest
    sq2 = math.sqrt(2) / 2
    Vs = torch.tensor([
        [0, 1],
        [1, 0],
        [sq2, sq2],
        [-sq2, sq2],
    ])
    
    # Evaluate everything
    F = surface.f(points)
    F_grad = surface.f_grad(points)
    Q_prime = [
        torch.sum(F_grad * V.expand_as(F_grad), dim=1)
        for V in Vs]

    # Plot
    norm = colors.SymLogNorm(linthresh=0.05, linscale=0.05, vmin=-20.0, vmax=20.0, base=10)
    
    ax_f.pcolormesh(X, Y, F.reshape(X.shape), cmap='RdBu_r', norm=norm, shading='auto')
    ax_f.set_title("F(x,y)")

    ax_grad_arg.pcolormesh(X, Y, np.arctan2(F_grad[:, 1], F_grad[:, 0]).reshape(X.shape), cmap='twilight')
    ax_grad_arg.set_title("arg(F(x,y))")

    ax_grad_norm.pcolormesh(X, Y, torch.linalg.norm(F_grad, dim=1).reshape(X.shape), cmap='RdBu_r', norm=norm, shading='auto')
    ax_grad_norm.set_title("||∇ F||")

    for i in range(len(Vs)):
        # F_grad . V
        axes_V[i].pcolormesh(X, Y, Q_prime[i].reshape(X.shape), cmap='RdBu_r', norm=norm, shading='auto')
        axes_V[i].set_title(f"[{Vs[i][0]:.2f} ; {Vs[i][1]:.2f}] . ∇F(x,y)")

        # init_t
        P = points
        V = Vs[i].expand_as(P)
        init_t = surface.init_newton(P, V)
        init_points = P + init_t.unsqueeze(1).expand_as(V) * V
        axes_V[i].scatter(init_points[:, 0], init_points[:, 1], s=1.0, c="black")
    del P
    del V
    
    for ax in axes.flat:
        ax.set_aspect("equal")
    f.tight_layout()
    f.suptitle(f"Implicit Surface {surface.__class__.__name__}")
    
    return f, axes


def newton_delta(surface, P, V, t):
    "Compute the delta for one step of Newton's method"

    dim = P.shape[1]
    points = P + t.unsqueeze(1).expand_as(V) * V
    F = surface.f(points)
    F_grad = surface.f_grad(points)
    denom = torch.sum(F_grad * V, dim=1)

    return F / denom
    

def newton_debug(surface, P, V, init_t):
    "Version of intersect_newton that returns every iterated t value"

    assert isinstance(P, Tensor) and P.dim() == 2
    assert isinstance(V, Tensor) and V.dim() == 2
    assert P.shape == V.shape
    dim = P.shape[1]
    assert dim == 2 or dim == 3
    assert init_t.shape[0] == P.shape[0], init_t.shape

    # Initialize solutions t
    t = init_t

    num_iter = 20
    t_history = torch.empty((P.shape[0], num_iter+1))

    t_history[:, 0] = init_t

    with torch.no_grad():
        for i in range(num_iter):
            # TODO warning if stopping due to max iter (didn't converge)
            delta = newton_delta(surface, P, V, t)
            # TODO early stop if delta is small enough
            t = t - 0.8*delta

            t_history[:, i+1] = t

    # One newton iteration for backwards pass
    t = t - newton_delta(surface, P, V, t)

    # t+1 steps, not including the final value
    return t, t_history


def gd_debug(surface, P, V, init_t):
    "Collision detection by gradient descent on F^2"

    assert isinstance(P, Tensor) and P.dim() == 2
    assert isinstance(V, Tensor) and V.dim() == 2
    assert P.shape == V.shape
    dim = P.shape[1]
    assert dim == 2 or dim == 3
    assert init_t.shape[0] == P.shape[0], init_t.shape

    # Initialize solutions t
    t = init_t

    num_iter = 20
    step_size = 0.1
    
    t_history = torch.empty((P.shape[0], num_iter+1))

    t_history[:, 0] = init_t

    for i in range(num_iter):
        dim = P.shape[1]
        points = P + t.unsqueeze(1).expand_as(V) * V
        F = surface.f(points)
        F_grad = surface.f_grad(points)
        dot = torch.sum(F_grad * V, dim=1)
        
        delta = 2*step_size*dot*F

        t = t - delta
        
        t_history[:, i+1] = t

    # t+1 steps, not including the final value
    return t, t_history


def Qplot(surface, P, V, collision_function=newton_debug):
    N = 1000
    tspace = torch.linspace(-4, 4, N)

    # t plot
    tpoints = P.unsqueeze(0).expand((N, 2)) + tspace.unsqueeze(1).expand((N, 2)) * V.unsqueeze(0).expand((N, 2))
    Q = surface.f(tpoints)
    Qgrad = torch.sum(surface.f_grad(tpoints) * V, dim=1)

    t_solve, t_history = collision_function(surface, P.unsqueeze(0), V.unsqueeze(0), init_t=torch.zeros((1,)))
    
    points_history = P.unsqueeze(0) + t_history[0, :].squeeze(0).unsqueeze(1).expand((-1, 2)) * V.unsqueeze(0)
    final_point = P + t_solve[0].expand((2)) * V
    
    fig, axes = plt.subplot_mosaic([['upper left', 'right'],
                                  ['lower left', 'right']],
                                   figsize=(10, 5), layout="constrained")
    ax_qplot, ax_splot, ax_history = axes.values()

    # t plot: plot Q and Q grad
    ax_qplot.plot(tspace.detach().numpy(), Q.detach().numpy(), label="Q(t)")
    ax_qplot.plot(tspace.detach().numpy(), Qgrad.detach().numpy(), label="Q'(t)")
    ax_qplot.grid()
    ax_qplot.set_xlabel("t")
    ax_qplot.legend()

    # t plot: plot t history
    ax_qplot.scatter(t_history[0, :], surface.f(points_history), c=range(t_history.shape[1]), cmap="viridis", marker="o")

    # Surface plot: plot F
    X, Y, ppoints = sample_grid((-10, 10), (-10, 10), 200)
    norm = colors.SymLogNorm(linthresh=0.05, linscale=0.05, vmin=-20.0, vmax=20.0, base=10)
    ax_splot.pcolormesh(X, Y, surface.f(ppoints).reshape(X.shape), cmap='RdBu_r', norm=norm, shading='auto')

    # Surface plot: plot the line
    ax_splot.plot(tpoints[:, 0], tpoints[:, 1], color="black", linewidth=1, marker="none")

    # Surface plot: points history
    ax_splot.scatter(points_history[:, 0], points_history[:, 1],  c=range(t_history.shape[1]), cmap="viridis", marker="+")

    # History plot: plot F
    ax_history.plot(range(t_history.shape[1]), surface.f(points_history), label="F(P+tV)")
    ax_history.legend()
    ax_history.set_xlabel("iteration")
    ax_history.set_title(f"final F = {surface.f(final_point.unsqueeze(0))[0].item():.6f}")
    
    ax_splot.set_title("F(x,y)")
    ax_splot.set_aspect("equal")



theta = math.radians(-40)
Qplot(tlm.Sphere(6, 3),
      P=torch.tensor([1.5, 3.], dtype=torch.float64),
      V=normalize(torch.tensor([math.cos(theta), math.sin(theta)], dtype=torch.float64), dim=0)
)

theta = math.radians(-40)
Qplot(tlm.Sphere(6, 3),
      P=torch.tensor([1.5, 3.], dtype=torch.float64),
      V=normalize(torch.tensor([math.cos(theta), math.sin(theta)], dtype=torch.float64), dim=0),
      collision_function=gd_debug
)

theta = math.radians(-40)
Qplot(tlm.Sphere(6, 3),
      P=torch.tensor([2.5, 1.], dtype=torch.float64),
      V=normalize(torch.tensor([math.cos(theta), math.sin(theta)], dtype=torch.float64), dim=0)
)

theta = math.radians(-40)
Qplot(tlm.Sphere(6, 3),
      P=torch.tensor([-2.5, 5.], dtype=torch.float64),
      V=normalize(torch.tensor([math.cos(theta), math.sin(theta)], dtype=torch.float64), dim=0)
)

theta = math.radians(-40)
Qplot(tlm.Sphere3(6, 3),
      P=torch.tensor([1.5, 3.], dtype=torch.float64),
      V=normalize(torch.tensor([math.cos(theta), math.sin(theta)], dtype=torch.float64), dim=0)
)

theta = math.radians(15.0)
Qplot(tlm.Sphere(6, 6),
      P=torch.tensor([2., 2.], dtype=torch.float64),
      V=normalize(torch.tensor([math.cos(theta), math.sin(theta)], dtype=torch.float64), dim=0)
)

theta = math.radians(45.0)
Qplot(tlm.Sphere(6, 6),
      P=torch.tensor([2., 2.], dtype=torch.float64),
      V=normalize(torch.tensor([math.cos(theta), math.sin(theta)], dtype=torch.float64), dim=0)
)

theta = math.radians(95.0)
Qplot(tlm.Sphere(6, 6),
      P=torch.tensor([2., 2.], dtype=torch.float64),
      V=normalize(torch.tensor([math.cos(theta), math.sin(theta)], dtype=torch.float64), dim=0)
)

theta = math.radians(95.0)
Qplot(tlm.Sphere(6, 6),
      P=torch.tensor([2., 2.], dtype=torch.float64),
      V=normalize(torch.tensor([math.cos(theta), math.sin(theta)], dtype=torch.float64), dim=0),
      collision_function=gd_debug
)

theta = math.radians(95.0)
Qplot(tlm.Sphere(6, 6),
      P=torch.tensor([-2., 2.], dtype=torch.float64),
      V=normalize(torch.tensor([math.cos(theta), math.sin(theta)], dtype=torch.float64), dim=0)
)

f, _ = surface_sdf_analysis(tlm.Sphere(6, 3))
f.savefig("sphere.png")

f, _ = surface_sdf_analysis(tlm.Sphere3(6, 3))
f.savefig("sphere2.png")

#f, _ = surface_sdf_analysis(tlm.Parabola(5, 0.08))
#f.savefig("parabola.png")

#plot_sdf(parabola)
#plot_sdf(circle)
#plot_sdf(circle2)

#print(tlm.Sphere(5, 5).extent_x())

#plot_sdf(box)
#plot_sdf_surface(tlm.Sphere2(3, 8))
#plot_sdf_surface(tlm.Sphere(5, 5))
#plot_sdf_surface(tlm.Sphere(3, 8))
#plot_sdf_surface(tlm.Sphere3(10, 8))


#plot_sdf_surface(tlm.Sphere(3, 9))
#plot_sdf_surface(tlm.Asphere(diameter=5, R=5, K=0, A4=0.))
#plot_sdf_surface(tlm.Asphere(diameter=20, R=-15, K=-1.2, A4=0.00045))
#plot_sdf_surface(tlm.Parabola(diameter=5, a=0.02))
